In [53]:
import numpy as np
import math
from scipy.linalg import expm
from numpy import linalg as LA
from functools import cache
from math import factorial as fact
from tqdm import tqdm

In [54]:
D_0 = np.matrix([
    [-0.405780, 0],
    [0, -0.013173]
])

In [55]:
D_1 = np.matrix([
    [0.403080, 0.0027],
    [0.007338, 0.005835]
])

In [56]:
matrix_size = 2
lambd = 0.05
gamma = 0.4
epsilon = 1e-6
b_1 = 1
K = 1
h = np.diag(-D_0).max()

In [57]:
#D_0 = np.matrix([-lambd])
#D_1 = np.matrix([lambd])

In [58]:
matrix_size = len(D_0)

In [59]:
O = np.zeros((matrix_size, matrix_size))

In [60]:
I = np.identity(matrix_size)

$$\varphi_k(t)$$

In [61]:
def varphi_k_t(k, t):
    return (gamma * t)**k / math.factorial(k) * math.exp(-gamma*t)

$$\hat\varphi_k(t)$$

In [62]:
def hat_varphi_k_t(k, t):
    norm = 1
    summa = 0
    i = 0
    while norm > epsilon:
        norm = varphi_k_t(k + i, t)
        summa += norm
        i += 1
    return summa

$$\varphi_k$$

In [63]:
def varphi_k(k):
    return varphi_k_t(k, b_1)

$$\hat\varphi_k$$

In [64]:
def hat_varphi_k(k):
    norm = 1
    summa = 0
    i = 0
    while norm > epsilon:
        norm = varphi_k(k + i)
        summa += norm
        i += 1
    return summa

$$\sum\limits_{i=0}^{\infty}P(i,t)z^i=e^{(D_0+D_1z)t}$$

In [65]:
def P_i_t_z(z, t):
    return expm((D_0 + D_1 * z) * t)

$$K_n^{(j)}$$

In [66]:
@cache
def K_n(n, j):
    if j == 0 and n == 0:
        return I.copy()
    elif j == 0 and n >= 1:
        return O.copy()
    elif n == 0:
        return K_n(0, j - 1) * (I + D_0 / h)
    elif n>=1 and j>=1:
        return K_n(n - 1, j-1) * D_1 / h + K_n(n, j-1) * (I + D_0 / h)


$$\Phi(i,k)$$

In [67]:
def Phi_i_k(i, k):
    norm = 1
    j = 0
    summa = 0
    while norm > epsilon or norm == 0:
        element = (h * b_1)**j / fact(j) * K_n(i, j)
        norm = LA.norm(element)
        summa += element
        j += 1
    return np.exp(-(h + gamma) * b_1) * (gamma * b_1)**k / fact(k) * summa

In [68]:
Phi_i_k(2, 5)

array([[3.09731967e-06, 2.41396506e-08],
       [6.56062060e-08, 1.45371380e-09]])

$$\hat\Phi(i,k)$$

In [69]:
def hat_Phi_i_k(i, k):
    # First summ
    norm = 1
    j = 0
    summa_1 = 0
    while norm > epsilon or norm == 0:
        element = (h * b_1)**j / fact(j) * K_n(i, j)
        norm = LA.norm(element)
        summa_1 += element
        j += 1
    # Second sum
    norm = 1
    l = k
    summa_2 = 0
    while norm > epsilon or norm == 0:
        element = (gamma * b_1)**l / fact(l)
        norm = LA.norm(element)
        summa_2 += element
        l += 1
    return np.exp(-(h + gamma) * b_1) * summa_1 * summa_2

In [102]:
hat_Phi_i_k(1, 1)

array([[0.0885637 , 0.00072655],
       [0.00197461, 0.00189851]])

$$N(m)$$

$$ N(m) =\frac{\gamma}{h+\gamma} \sum\limits_{j=0}^{\infty}(\frac{h}{h+\gamma})^jK_m^{(j)},\;$ $m \ge 0$$

In [103]:
def N(m):
    norm = 1
    j = 0
    summa = 0
    while norm > epsilon or norm == 0:
        element = (h / (h + gamma))**j * K_n(m, j)
        norm = LA.norm(element)
        summa += element
        j += 1
    return gamma / (h + gamma) * summa

In [71]:
N(0)

array([[0.49641341, 0.        ],
       [0.        , 0.96811747]])

$$M(r)$$

In [72]:
def M(r):
    return gamma**r * (-D_0 + gamma * I)**(-(r + 1)) * D_1

$$\hat M(r)$$

In [73]:
def hat_M(r):
    return gamma**r * (-D_0 + gamma * I)**(-r) * (-D_0)**(-1) * D_1

$$P\left\{(0, 0) \to (j, k' ) \right\}=M(0)\sum\limits_{n = 0}^{j}N(n)\Phi(j-n, k') + N(0)\sum\limits_{m = 0}^{k'}M(m)\Phi(j, k' - m),$$
$$\; k'= {\overline{0,K-2}};$$

In [74]:
def P_0_0_j_k_(j, k_):
    summa_1 = np.zeros((matrix_size, matrix_size))
    summa_2 = np.zeros((matrix_size, matrix_size))
    for n in range(j + 1):
        summa_1 += N(n) * Phi_i_k(j - n, k_)
    for m in range(k_ + 1):
        summa_2 += M(m) * Phi_i_k(j, k_ - m)
    return M(0) * summa_1 + N(0) * summa_2    

In [75]:
P_0_0_j_k_(2, 5)

matrix([[6.13220679e-04, 2.71789647e-10],
        [1.79241337e-07, 6.55234036e-06]])


$$P\left\{(0, 0) \to (j,K-1 ) \right\}= N(0)\biggl[\sum\limits_{m = 0}^{K-1}M(m)\Phi(j, K-1 - m)
+\hat {M}(K) \Phi(j, 0)\biggr] $$$$+M(0)\sum\limits_{n = 0}^{j}N(n)\Phi(j-n, K-1);$$

In [76]:
def P_0_0_j_K_1(j):
    summa_1 = np.zeros((matrix_size, matrix_size))
    summa_2 = np.zeros((matrix_size, matrix_size))
    for m in range(K):
        summa_1 += M(m) * Phi_i_k(j, K - 1 - m)
    summa_1 += hat_M(K) * Phi_i_k(j, 0)

    for n in range(j + 1):
        summa_2 += N(n) * Phi_i_k(j - n, K - 1)

    return N(0) * summa_1 + M(0) * summa_2

In [77]:
P_0_0_j_K_1(1)

matrix([[1.89018709e-01, 4.28292801e-05],
        [3.55781268e-03, 2.63251671e-03]])

$$P\left\{(0, 0) \to (j, K) \right\}= N(0)\biggl[\sum\limits_{m = 0}^{K-1}M(m)\hat\Phi(j, K - m) + \hat M(K)\hat \Phi(j, 1)\biggr]
$$$$+M(0)\sum\limits_{n = 0}^{j}N(n)\hat\Phi(j-n, K);$$



In [78]:
def P_0_0_j_K(j):
    summa_1 = np.zeros((matrix_size, matrix_size))
    summa_2 = np.zeros((matrix_size, matrix_size))
    for m in range(K):
        summa_1 += M(m) * hat_Phi_i_k(j, K - m)
    summa_1 += hat_M(K) * hat_Phi_i_k(j, 1)

    for n in range(j + 1):
        summa_2 += N(n) * hat_Phi_i_k(j - n, K)

    return N(0) * summa_1 + M(0) * summa_2

In [79]:
P_0_0_j_K(1)

matrix([[9.29640694e-02, 2.10644977e-05],
        [1.74982015e-03, 1.29473673e-03]])

$$P\left\{(0, k) \to (j, k') \right\}=\sum\limits_{m = 0}^{k'-k+1}M(m)\Phi(j,k' - k + 1 - m),\;  k=\overline{1,K}, k' =\overline{k-1, {K-2}};$$

In [80]:
def P_0_k_j_k_(j, k, k_):
    summa_1 = np.zeros((matrix_size, matrix_size))
    for m in range(k_ - k + 2):
        summa_1 += M(m) * Phi_i_k(j, k_ - k + 1 - m)
    return summa_1

In [81]:
P_0_k_j_k_(1,1,1)

array([[0.08075946, 0.00067403],
       [0.00445294, 0.00011048]])

$$P\left\{(0, k) \to (j, K-1) \right\}=\sum\limits_{m = 0}^{K-k}M(m)\Phi(j,K-k - m)+
\hat{M}(K-k+1)\Phi(j,0),\; k=\overline{1,K};
$$

In [82]:
def P_0_k_j_K_1(j, k):
    summa_1 = np.zeros((matrix_size, matrix_size))
    for m in range(K - k + 1):
        summa_1 += M(m) * Phi_i_k(j, K - k - m)
    summa_1 += hat_M(K - k + 1) * Phi_i_k(j, 0)
    return summa_1

In [83]:
P_0_k_j_K_1(1,1)

array([[0.17890023, 0.00149312],
       [0.10208706, 0.00253276]])

$$P\left\{(0, k) \to (j, K)\right\}=\sum\limits_{m = 0}^{K-k}M(m) \hat \Phi(j, K - k + 1 - m) + \hat M(K - k + 1)\hat \Phi(j, 1),\;  k =\overline{1,K};$$

In [84]:
def P_0_k_j_K(j, k):
    summa_1 = np.zeros((matrix_size, matrix_size))
    for m in range(K - k + 1):
        summa_1 += M(m) * hat_Phi_i_k(j, K - k + 1 - m)
    summa_1 += hat_M(K - k + 1) * hat_Phi_i_k(j, 1)
    return summa_1

In [85]:
P_0_k_j_K(1, 2)

array([[0.08798755, 0.00073435],
       [0.05020894, 0.00124567]])

$$P\left\{(i, 0) \to (j, k')  \right\}= \sum\limits_{m = 0}^{j - i + 1}N(m)\Phi(j - i + 1 - m, k'),\; i\geq 1,\;  k' =\overline{0, K-1}; $$

In [86]:
def P_i_0_j_k_(i, j, k_):
    summa_1 = np.zeros((matrix_size, matrix_size))
    for m in range(j - i + 2):
        summa_1 += N(m) * Phi_i_k(j - i + 1 - m, k_)
    return summa_1

In [87]:
P_i_0_j_k_(1,1,1)

array([[0.0801304 , 0.        ],
       [0.        , 0.00511274]])

$$P\left\{(i, 0) \to (j, K) \right\}=\sum\limits_{m = 0}^{j - i + 1}N(m)\hat \Phi(j - i + 1 - m, K),\; i \geq 1; $$

In [88]:
def P_i_0_j_K(i, j):
    summa_1 = np.zeros((matrix_size, matrix_size))
    for m in range(j - i + 2):
        summa_1 += N(m) * hat_Phi_i_k(j - i + 1 - m, K)
    return summa_1

In [89]:
P_i_0_j_K(1,1)

array([[0.09852527, 0.        ],
       [0.        , 0.00628643]])


$$P\left\{(i, k) \to (j, k') \right\} =\Phi(j - i + 1, k' - k+1),\; i \geq 1,\;  k' =\overline{k-1, K-1},\; k=\overline{1,K};$$

In [90]:
def P_i_k_j_k_(i, j, k, k_):
    return Phi_i_k(j - i + 1, k_ - k + 1)

In [91]:
P_i_k_j_k_(1,1,1,1)

array([[0.07202868, 0.0005909 ],
       [0.00160595, 0.00154405]])

$$P\left\{(i, k) \to (j, K)\right\}= \hat \Phi(j - i + 1, K - k+1),\; i \geq 1,\;  k=\overline{1,K}.$$

In [92]:
def P_i_k_j_K(i, j, k):
    return hat_Phi_i_k(j - i + 1, K - k + 1)

In [93]:
P_i_k_j_K(1,1,1)

array([[0.0885637 , 0.00072655],
       [0.00197461, 0.00189851]])

In [94]:
P = np.zeros((K + 1, K + 1))

In [95]:
def V(j_1):
    block = []
    for i in range(K + 1):
        row = []
        for j in range(K + 1):
            if i == 0 and j <= K - 2:        
                row.append(P_0_0_j_k_(j_1, j))
            elif i == 0 and j == K - 1:
                row.append(P_0_0_j_K_1(j_1))
            elif i == 0 and j == K:
                row.append(P_0_0_j_K(j_1)) 
            elif i > 0 and j >= i - 1 and j <= K - 2:
                row.append(P_0_k_j_k_(j_1, i, j))
            elif i > 0 and j == K - 1:
                row.append(P_0_k_j_K_1(j_1, i))
            elif i > 0 and j == K:
                row.append(P_0_k_j_K(j_1, i))
        block.append(row)
    return np.block(block)

In [104]:
a = V(0).sum(axis=1)
for x in range(1, 20):
    a += V(x).sum(axis=1)
a

[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0

 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[

 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[

 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[

matrix([[0.99069193],
        [0.45851318],
        [1.        ],
        [1.        ]])

$$ \sum\limits_{j=0}^\infty \sum\limits_{k=0 }^K  P{(0,0)->(j,k)}$$  $$\sum\limits_{j=0}^\infty \sum\limits_{k=0 }^K  P{(0,1)->(j,k)} $$

In [97]:
j_1 = 1
block = []
for i in range(K + 1):
    row = []
    for j in range(K + 1):
        if i == 0 and j <= K - 2:
            P[i][j] = 1
            row.append(P_0_0_j_k_(j_1, j))
        elif i == 0 and j == K - 1:
            P[i][j] = 2
            row.append(P_0_0_j_K_1(j_1))
        elif i == 0 and j == K:
            P[i][j] = 3
            row.append(P_0_0_j_K(j_1))
        elif i > 0 and j >= i - 1 and j <= K - 2:
            P[i][j] = 4
            row.append(P_0_k_j_k_(j_1, i, j))
        elif i > 0 and j == K - 1:
            P[i][j] = 5
            row.append(P_0_k_j_K_1(j_1, i))
        elif i > 0 and j == K:
            P[i][j] = 6
            row.append(P_0_k_j_K(j_1, i))
        else:
            P[i][j] = 0
            row.append(O)
    block.append(row)
P

array([[2., 3.],
       [5., 6.]])

In [98]:
j_1 = 2
block = []
for i in range(K + 1):
    row = []
    for j in range(K + 1):
        if i == 0 and j <= K - 2:
            row.append(f'{i} {j}')
        elif i == 0 and j == K - 1:
            row.append(f'{i} {j}')
        elif i == 0 and j == K:
            row.append(f'{i} {j}')
        elif i > 0 and j >= i - 1 and j <= K - 2:
            row.append(f'{i} {j}')
        elif i > 0 and j == K - 1:
            row.append(f'{i} {j}')
        elif i > 0 and j == K:
            row.append(f'{i} {j}')
        else:
            row.append(' 0 ')
    block.append(row)
block

[['0 0', '0 1'], ['1 0', '1 1']]

In [99]:
def ksis():
    ksis = np.zeros(K)
    ksis[0] = 1
    varphi_0 = varphi_k(0)

    for k in range(1, K):
        summ = np.sum(np.array([ksis[i] * varphi_k(k + 1 - i) for i in range(1, k)]))
        ksis[k] = (ksis[k-1] - varphi_k(k) - summ) / varphi_0
    return ksis

In [100]:
def ergodicity():
    u_0 = 1 / ksis().sum()
    return lambd * b_1 + u_0 * lambd / gamma

In [101]:
ergodicity()

0.175